In [1]:
!nvidia-smi

Sun Jun 26 23:55:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:4F:00.0 Off |                    0 |
| N/A   31C    P0    53W / 300W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install unseal -q

In [3]:
!pip install transformers -q

In [4]:
from typing import Union, Tuple, Optional

import matplotlib.pyplot as plt
from matplotlib import cm
from statistics import stdev
from matplotlib.colors import BoundaryNorm
import numpy as np
import random
import json
import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import unseal
from unseal.hooks import Hook, HookedModel, common_hooks
from unseal.transformers_util import load_from_pretrained, get_num_layers

import sys
sys.path.append("../../../lib")
import utility
import hooks

In [5]:
unhooked_model, tokenizer, config = load_from_pretrained('EleutherAI/gpt-j-6B')

In [6]:
model = HookedModel(unhooked_model)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(f'Model loaded and on device {device}!')

Model loaded and on device cuda!


In [7]:
# layer names
print(list(model.layers.keys()))

['transformer', 'transformer->wte', 'transformer->drop', 'transformer->h', 'transformer->h->0', 'transformer->h->0->ln_1', 'transformer->h->0->attn', 'transformer->h->0->attn->attn_dropout', 'transformer->h->0->attn->resid_dropout', 'transformer->h->0->attn->k_proj', 'transformer->h->0->attn->v_proj', 'transformer->h->0->attn->q_proj', 'transformer->h->0->attn->out_proj', 'transformer->h->0->mlp', 'transformer->h->0->mlp->fc_in', 'transformer->h->0->mlp->fc_out', 'transformer->h->0->mlp->act', 'transformer->h->0->mlp->dropout', 'transformer->h->1', 'transformer->h->1->ln_1', 'transformer->h->1->attn', 'transformer->h->1->attn->attn_dropout', 'transformer->h->1->attn->resid_dropout', 'transformer->h->1->attn->k_proj', 'transformer->h->1->attn->v_proj', 'transformer->h->1->attn->q_proj', 'transformer->h->1->attn->out_proj', 'transformer->h->1->mlp', 'transformer->h->1->mlp->fc_in', 'transformer->h->1->mlp->fc_out', 'transformer->h->1->mlp->act', 'transformer->h->1->mlp->dropout', 'transf

In [8]:
# load data
f = open("../../../datasets/known_1000.json")
prompts = json.load(f)

In [9]:
# prepare data
texts = [(prompts[i]['subject'], prompts[i]['prompt'] + ' ' + prompts[i]['attribute']) for i in range(len(prompts))]
print(texts[0])

('Vinson Massif', 'Vinson Massif is located in the continent of Antarctica')


In [10]:
### Batching requires inputs to be of the same size so we put each prompt in 
### a bucket depending on the number of tokens it contains.

inputs = [[] for z in range(30)] ## use 30 as an upper bound on the number of tokens
for subject, text in random.sample(texts, 1000):
    encoded_text, target_id = utility.prepare_input(text, tokenizer, device)
    num_tokens = encoded_text['input_ids'].shape[1]
    
    print("num tokens: ", num_tokens, "text: ", text)
    
    # get tokens
    input_ids = encoded_text['input_ids'][0]
    tkens = [tokenizer.decode(input_ids[i]) for i in range(len(input_ids))]
    
    # get position of subject tokens
    subject_positions = utility.get_subject_positions(tkens, subject)
    indx_str = str(subject_positions[0]) + ":" + str(subject_positions[-1]+1)
    
    # add star to subject tokens to later do analysis
    start, end = subject_positions[0], subject_positions[1]
    while start < end:
        tkens[start] = tkens[start] + "*"
        start += 1
    
    # save uncorrupted states for patching
    num_layers = get_num_layers(model, layer_key_prefix='transformer->h')
    output_hooks_names = [f'transformer->h->{layer}' for layer in range(num_layers)]
    output_hooks = [Hook(name, utility.save_output, name) for name in output_hooks_names]
    model(**encoded_text, hooks=output_hooks)
    
    hidden_states = [model.save_ctx[f'transformer->h->{layer}']['output'][0].detach() for layer in range(num_layers)] # need detach
    
    # get probability with uncorrupted subject
    p = model(**encoded_text, hooks=[])['logits'].softmax(dim=-1)[0,-1,target_id].item()
    
    NUM_RUNS = 10 # ROME runs each input 10 times
    for i in range(NUM_RUNS):
        # add noise hook
        seed = np.random.randint(100, size=1)[0]
        noise_hook = Hook(
            layer_name='transformer->wte',
            func=hooks.additive_output_noise(indices=indx_str, mean=0, std=0.025, seed=seed),
            key='embedding_noise',
        )
    
        # get probability with corrupted subject
        p_star = model(**encoded_text, hooks=[noise_hook])['logits'].softmax(dim=-1)[0,-1,target_id].item()
        
        inputs[num_tokens].append((num_tokens, num_layers, encoded_text, hidden_states, tkens, text, target_id, indx_str, seed, p, p_star))


num tokens:  12 text:  Ignatius of Antioch, who has the position of a bishop
num tokens:  10 text:  B-47 Stratojet is developed by the Boeing
num tokens:  14 text:  Pius VIII, whose position is that of a "superior" pope
num tokens:  8 text:  Seiyu Group's headquarters are in Tokyo
num tokens:  9 text:  Otto Hauser worked in the city of Berlin
num tokens:  13 text:  The language of L'Histoire was not the language of the French
num tokens:  5 text:  Philippines's capital, Manila
num tokens:  8 text:  Alfa Romeo 155, produced by Fiat
num tokens:  11 text:  Annise Parker holds the title of the first openly gay mayor
num tokens:  6 text:  2005 Australian Open is located in Melbourne
num tokens:  9 text:  James Northcote died in the city of London
num tokens:  6 text:  Jonathan Pearce is employed by the BBC
num tokens:  10 text:  Madame de Montesson died in the city of Paris
num tokens:  11 text:  Nonchan Noriben was created in the country of Japan
num tokens:  7 text:  Eavan Boland was born

num tokens:  10 text:  Matias Kupiainen was born in Helsinki
num tokens:  7 text:  The Ed Sullivan Show premieres on CBS
num tokens:  8 text:  air traffic control works in the field of aviation
num tokens:  5 text:  Paul McCartney, playing the piano
num tokens:  9 text:  Arisaka was created in the country of Japan
num tokens:  8 text:  Yangtze belongs to the continent of Asia
num tokens:  7 text:  Second Polish Republic's capital city is Warsaw
num tokens:  10 text:  Yannick Cahuzac plays as a central midfielder
num tokens:  4 text:  Jeff Zucker, of CNN
num tokens:  11 text:  The original language of Ma vie en rose is a French
num tokens:  11 text:  The location of 2000 Australian Open is in the heart of Melbourne
num tokens:  7 text:  Core 2, a product developed by Intel
num tokens:  9 text:  Lake Vostok belongs to the continent of Antarctica
num tokens:  6 text:  Windows Live Mesh is developed by Microsoft
num tokens:  11 text:  Dannii Minogue was created in the country of Australia


num tokens:  6 text:  Wii U is developed by Nintendo
num tokens:  4 text:  Flickr is owned by Yahoo
num tokens:  16 text:  Abdalqadir as-Sufi is affiliated with the religion of Islam
num tokens:  7 text:  In Panama, the language spoken is Spanish
num tokens:  11 text:  Leonard Peikoff works in the area of the philosophy
num tokens:  9 text:  The native language of Bernard Giraudeau is French
num tokens:  10 text:  Hurricane Ridge is a part of the continent of Antarctica
num tokens:  9 text:  The native language of Victor Prosper Considerant is French
num tokens:  8 text:  Delmarva Peninsula was named for the Delaware
num tokens:  5 text:  The Good Wife debuted on CBS
num tokens:  9 text:  Grand Duchy of Finland's capital, Helsinki
num tokens:  7 text:  Paul Otellini is employed by Intel
num tokens:  9 text:  Maryland Film Festival can be found in the Baltimore
num tokens:  8 text:  De-Phazz plays a lot of jazz
num tokens:  12 text:  Acca of Hexham holds the position of the first female

num tokens:  6 text:  The Bill Cosby Show debuted on NBC
num tokens:  10 text:  Suite Habana was created in the country of Cuba
num tokens:  12 text:  Kalamazoo County can be found in the southeast corner of Michigan
num tokens:  6 text:  Melodiya is headquartered in Moscow
num tokens:  17 text:  Raina Kabaivanska is known for performing in the world's most famous opera
num tokens:  10 text:  Mohsen Mirdamadi is a citizen of Iran
num tokens:  12 text:  The original language of Die Nibelungen was written in German
num tokens:  7 text:  Osman I follows the religion of Islam
num tokens:  11 text:  Amos Tversky works in the area of cognitive psychology
num tokens:  8 text:  L'Aurore is written in French
num tokens:  13 text:  Hope and Anchor, Islington is located in the heart of London
num tokens:  9 text:  Godrej Consumer Products Limited is headquartered in Mumbai
num tokens:  13 text:  Giacomo Barozzi da Vignola passed away in Rome
num tokens:  11 text:  Edward Teller's expertise is in 

num tokens:  11 text:  Etobicoke North is located in the country of Canada
num tokens:  9 text:  Maurice Richard professionally plays the sport of hockey
num tokens:  13 text:  In Canton of Fribourg, the language spoken is French
num tokens:  10 text:  Vinson Massif is located in the continent of Antarctica
num tokens:  9 text:  Scooby Doo was originally aired on CBS
num tokens:  17 text:  Rosenberg Trio is known for performing a variety of styles of music, including jazz
num tokens:  5 text:  Roman Empire's capital, Constantinople
num tokens:  10 text:  Petros Voulgaris is a citizen of Greece
num tokens:  10 text:  Shigeki Maruyama is a citizen of Japan
num tokens:  6 text:  Sunday Night Baseball premieres on ESPN
num tokens:  20 text:  The original language of Kadhalil Sodhappuvadhu Yeppadi was written in the Tamil
num tokens:  17 text:  Odnoklassniki was written in the early 1990s by a group of Russian
num tokens:  7 text:  Le Mauricien is written in French
num tokens:  10 text:  Sh

num tokens:  9 text:  The native language of Lisa Ekdahl is Swedish
num tokens:  7 text:  Wii Balance Board is produced by Nintendo
num tokens:  8 text:  Sesamstraat is written in Dutch
num tokens:  9 text:  In Raseborg, the language spoken is Swedish
num tokens:  9 text:  X11.app, a product developed by Apple
num tokens:  9 text:  The native language of Nicolaas Pierson is Dutch
num tokens:  12 text:  Federal Palace of Switzerland is in the heart of the city of Bern
num tokens:  10 text:  Jan Swammerdam died in the city of Amsterdam
num tokens:  4 text:  Italy's capital, Rome
num tokens:  8 text:  Mercure de France's headquarters are in Paris
num tokens:  15 text:  Alexander Knaifel is known for performing in the world's most prestigious opera
num tokens:  10 text:  Sjogren Glacier belongs to the continent of Antarctica
num tokens:  10 text:  Cairo American College is located in the country of Egypt
num tokens:  7 text:  Show Me a Hero premieres on HBO
num tokens:  17 text:  The genre

num tokens:  8 text:  Cosimo Fancelli was born in Rome
num tokens:  11 text:  The original language of Les Francs-juges is French
num tokens:  8 text:  Pat Tillman professionally plays the sport of football
num tokens:  14 text:  The New School for Jazz and Contemporary Music is located in the heart of Manhattan
num tokens:  16 text:  Postcards Records is known for performing a wide variety of music, from rock to jazz
num tokens:  18 text:  Peter Josef von Lindpaintner is known for performing the first ever performance of the opera
num tokens:  10 text:  The native language of Lee Chang-dong is Korean
num tokens:  8 text:  Monster Rancher, that originated in Japan
num tokens:  6 text:  Stephen Elop is employed by Microsoft
num tokens:  9 text:  Hohenlohe, in the south of Germany
num tokens:  8 text:  Mount Foster is located in the continent of Antarctica
num tokens:  6 text:  Hotel Room premieres on HBO
num tokens:  11 text:  The native language of Louis-Nicolas Davout is French
num to

num tokens:  9 text:  The Famous Teddy Z was originally aired on the CBS
num tokens:  7 text:  Chromebook Pixel is developed by Google
num tokens:  6 text:  Law & Order was released on NBC
num tokens:  7 text:  Eiko Koike is originally from Tokyo
num tokens:  6 text:  Windows Media Player's owner, Microsoft
num tokens:  8 text:  Nicosia is a twin city of Athens
num tokens:  11 text:  In history of Limousin, the language spoken is French
num tokens:  8 text:  Polish Socialist Party's headquarters are in Warsaw
num tokens:  8 text:  Daiki Arioka's profession is an actor
num tokens:  10 text:  Now Kandeh District is located in the country of Iran
num tokens:  9 text:  Debenham Glacier belongs to the continent of Antarctica
num tokens:  10 text:  Muhammad Asad is affiliated with the religion of Islam
num tokens:  9 text:  Mainichi Shinbun, that originated in Japan
num tokens:  11 text:  Marcellin Berthelot died in the city of Paris
num tokens:  18 text:  Frans Hubert Edouard Arthur Walter 

num tokens:  8 text:  Commonwealth of the Philippines's capital, Manila
num tokens:  7 text:  Hungarian Soviet Republic's capital, Budapest
num tokens:  7 text:  Coco Chanel passed away in Paris
num tokens:  10 text:  Alasdair Milne is employed by the BBC
num tokens:  8 text:  Ryan Smyth professionally plays the sport of hockey
num tokens:  7 text:  Drake Britton plays as the pitcher
num tokens:  15 text:  The original language of Celia en el colegio is a mixture of Spanish
num tokens:  8 text:  Irish Free State belongs to the continent of Europe
num tokens:  8 text:  Eiko Shimamiya is a citizen of Japan
num tokens:  7 text:  The Sports Reporters premieres on ESPN
num tokens:  7 text:  Eddy Cue is employed by Apple
num tokens:  17 text:  Elizabeth Butler-Sloss, Baroness Butler-Sloss worked in the city of London
num tokens:  16 text:  The language of Amar en tiempos revueltos is a mixture of Spanish
num tokens:  7 text:  Deobandi follows the religion of Islam
num tokens:  6 text:  Lexus

In [11]:
# statistics on number of prompts for each length tokens
for i in range(len(inputs)):
    length = len(inputs[i])
    if length != 0:
        print("num tokens: ", i, "number of prompts: ", int(length/NUM_RUNS))

num tokens:  4 number of prompts:  13
num tokens:  5 number of prompts:  38
num tokens:  6 number of prompts:  95
num tokens:  7 number of prompts:  118
num tokens:  8 number of prompts:  130
num tokens:  9 number of prompts:  140
num tokens:  10 number of prompts:  134
num tokens:  11 number of prompts:  101
num tokens:  12 number of prompts:  67
num tokens:  13 number of prompts:  48
num tokens:  14 number of prompts:  44
num tokens:  15 number of prompts:  21
num tokens:  16 number of prompts:  13
num tokens:  17 number of prompts:  19
num tokens:  18 number of prompts:  12
num tokens:  19 number of prompts:  2
num tokens:  20 number of prompts:  2
num tokens:  23 number of prompts:  1
num tokens:  25 number of prompts:  1
num tokens:  27 number of prompts:  1


In [12]:
# remove input buckets with 0 prompts
inputs = [inp for inp in inputs if inp != []]

In [13]:
# deal with longest prompts first :)
inputs.reverse()

In [14]:
results = {}
results_perc_impr = {}
batch_schedule = {i : int(1000/i) for i in range(1, 30)}
for inp in inputs:
    print(f"Number of prompts in this bucket: {len(inp)}.")
    
    batch_size = batch_schedule[inp[0][0]]
    batched_data = DataLoader(inp, batch_size=batch_size)
    for data in batched_data:
        curr_batch_size = data[0].size()[0]
        print("This batch has", curr_batch_size, "examples.")
        
        num_tokens = data[0][0].item()
        num_layers = data[1][0].item()
        encoded_text = data[2]
        hidden_states = data[3]
        tkens = [[t[i] for t in data[4]] for i in range(curr_batch_size)]
        text = [t for t in data[5]]
        target_ids = [t.item() for t in data[6]]
        indx_strs = [s for s in data[7]]
        seeds = [seed.item() for seed in data[8]]
        p = [p.item() for p in data[9]]
        p_stars = [p_s.item() for p_s in data[10]]

        noise_hooks = [] # can probably be batched [will take a look later on]
        for i in range(curr_batch_size):
            noise_hook = Hook(
                layer_name='transformer->wte',
                func=hooks.additive_output_noise(indices=indx_strs[i], mean=0, std=0.1, index=i, seed=seeds[i]),
                key='embedding_noise',
            )
            noise_hooks.append(noise_hook)
            
        for layer in tqdm(range(num_layers)):
            for position in range(num_tokens):
                hook = Hook(
                    layer_name=f'transformer->h->{layer}',
                    func= hooks.hidden_patch_hook_fn(layer, position, hidden_states),
                    key=f'patch_{layer}_pos{position}'
                )
                output = model(**encoded_text, hooks=noise_hooks+[hook])
                for i in range(curr_batch_size):
                    if text[i] not in results:
                        accessory = (text[i], tkens[i], num_layers, sum(p) / len(p), sum(p_stars) / len(p_stars))
                        results[text[i]] = (torch.zeros((num_tokens, num_layers)), accessory)
                        results_perc_impr[text[i]] = (torch.zeros((num_tokens, num_layers)), accessory)
                    p_star_h = torch.softmax(output["logits"][i][0,-1,:], 0)[target_ids[i]].item()
                    results[text[i]][0][position, layer] += p_star_h - p_stars[i]
                    if (p[i] - p_stars[i]) != 0:
                        results_perc_impr[text[i]][0][position, layer] += (p_star_h - p_stars[i]) / abs(p[i] - p_stars[i])

Number of prompts in this bucket: 10.
This batch has 10 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

Number of prompts in this bucket: 10.
This batch has 10 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

Number of prompts in this bucket: 10.
This batch has 10 examples.


...

  0%|          | 0/28 [00:00<?, ?it/s]

This batch has 180 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

Number of prompts in this bucket: 130.
This batch has 130 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

In [15]:
# Need to divide by NUM_RUNS
res = []
for _, value in results.items():
    result, accessory = value
    result = result / NUM_RUNS
    res.append((result, accessory))
    
res_perc_impr = []
for _, value in results_perc_impr.items():
    result, accessory = value
    result = result / NUM_RUNS
    res_perc_impr.append((result, accessory))

In [16]:
torch.save(res, "../data/indirect_effect_1000_examples.pt")
torch.save(res_perc_impr, "../data/percent_improvement_1000_examples.pt")